## US Financial Health
***

In this project provided by CodeAcademy we will importing various types of financial data to try and determine the financial health and volatility of the US between 1999 and 2019.

We’ll use the techniques we’ve learned for importing financial data, to import stock and commodity pricing data from csv files and the FRED API. Then grab GDP and goods and services export data from the World Bank API.

Finally, we’ll find the log returns of the imported data, and use that to determine the volatility of the data over the 20 year period.

In [26]:
# Importing modules
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader.wb as wb
from datetime import datetime

# Loading in data
gold_prices = pd.read_csv('gold_prices.csv')
crude_oil_prices = pd.read_csv('crude_oil_prices.csv')

In [27]:
# Viewing gold prices
gold_prices.head()

,Date,Gold_Price
0,2019-08-30,1528.40
1,2019-08-29,1540.20
2,2019-08-28,1537.15
3,2019-08-27,1532.95
4,2019-08-26,1503.80


In [28]:
# Viewing crude oil prices
crude_oil_prices.head()

,Date,Crude_Oil_Price,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Sep 11, 2018",69.25,NaN,NaN,NaN,NaN,NaN
1,"Sep 10, 2018",67.54,NaN,NaN,NaN,NaN,NaN
2,"Sep 07, 2018",67.75,NaN,NaN,NaN,NaN,NaN
3,"Sep 06, 2018",67.77,NaN,NaN,NaN,NaN,NaN
4,"Sep 05, 2018",68.72,NaN,NaN,NaN,NaN,NaN


In [34]:
# Now commodity prices have been imported we will import historical stock prices

# Defining start and end
start = datetime(1999, 1, 1) # Year, Month, Day
end = datetime(2019, 1, 1) # Year, Month, Day

# Getting historical prices for the NASDAQ100 and S&P500 using FRED API
nasdaq_data = web.DataReader('NASDAQ100', 'fred', start, end)
sap_data = web.DataReader('SP500', 'fred', start, end)

print(nasdaq_data.head())
print('\n')
print(sap_data.head())


            NASDAQ100
DATE                 
1999-01-01        NaN
1999-01-04    1854.39
1999-01-05    1903.00
1999-01-06    1963.95
1999-01-07    1966.35


              SP500
DATE               
2012-02-08  1349.96
2012-02-09  1351.95
2012-02-10  1342.64
2012-02-13  1351.77
2012-02-14  1350.50


In [30]:
# Importing high level economic data from the World Bank API
gdp_data = wb.download(indicator='NY.GDP.MKTP.CD', country=['US'], start=start, end=end)

# Retrieving high level export data for goods and services
export_data = wb.download(indicator='NE.EXP.GNFS.CN', country=['US'], start=start, end=end)

#### Calculating Log Return

At this point, we’ve imported all the data we need. But it’s all stored as either daily or yearly dollar amounts.

Pricing data is useful, but in this case, since we want to compare each data set, it would be even better if instead of daily/yearly pricing, we had information on the log returns from the daily/yearly prices.

In [37]:
# Defining our log_returns function
def log_return(prices):
    return np.log(prices / prices.shift(1))

# Getting log returns for our datasets
gold_returns = log_return(gold_prices['Gold_Price'])
crudeoil_returns = log_return(crude_oil_prices['Crude_Oil_Price'])
nasdaq_returns = log_return(nasdaq_data['NASDAQ100'])
sap_returns = log_return(sap_data['SP500'])
gdp_returns = log_return(gdp_data['NY.GDP.MKTP.CD'])
export_returns = log_return(export_data['NE.EXP.GNFS.CN'])



In [39]:
# Comparing volatility
print('Gold:', gold_returns.var())
print('Oil:', crudeoil_returns.var())
print('Nasdaq:', nasdaq_returns.var())
print('S&P500:', sap_returns.var())
print('GDP:', gdp_returns.var())
print('Export:', export_returns.var())

Gold: 0.00011375928671558533
Oil: 0.0005563207795629894
Nasdaq: 0.0003178379833057234
S&P500: 6.513399801482707e-05
GDP: 0.00034121401492647786
Export: 0.006247875993777988


Gold, notorious for being a stable investment has the second smallest variance.

Crude oil is the most volatile as gas prices are often unpredictable, especially in the last 20 years.

The NASDAQ 100 is more volatile than the S&P 500, this is due to the S&P 500 is far more diversified and tracks more of the market.

Then finally we have GDP and exports.

Exports are very volatile, which could have to do with industries moving overseas in the last 20 years, and global competition for the production of goods.